In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, matthews_corrcoef, f1_score, fbeta_score
from sklearn.impute import SimpleImputer

In [13]:
fault_diag = pd.read_csv('../data/fault_diag.csv')

/var/folders/xm/vmtk5pms0jzb2kmtm00lwtn80000gn/T/ipykernel_54025/699357233.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  fault_diag = pd.read_csv('../data/fault_diag.csv')


In [38]:
fault_diag.head()

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,...,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,Speed,Throttle,TurboBoostPressure,spn_fmi
0,1038243,55748536,2018-07-20 09:31:33,High (Severity Medium) J1939 Network #2,49,1231,16,True,2,105406655,...,12311.606397,1.651080,NaN,True,138.2,255.0,5.203984,100.0,5.80,1231_16
1,366301,7171498,2016-01-31 07:12:25,NaN,11,629,12,True,127,105301976,...,NaN,NaN,NaN,True,NaN,1279.0,NaN,NaN,NaN,629_12
2,1141606,81618595,2019-03-27 08:10:52,Low (Severity Medium) Engine Coolant Level,0,111,18,True,1,105338729,...,14987.537198,1.083109,NaN,True,59.0,2047.0,0.000000,100.0,0.29,111_18
3,1142275,81843663,2019-03-29 06:08:56,Low (Severity Medium) Engine Coolant Level,0,111,18,True,3,105338729,...,15098.753632,0.990648,NaN,True,73.4,2047.0,0.000000,100.0,0.00,111_18
4,1149647,83749925,2019-04-16 12:27:33,Low (Severity Medium) Engine Coolant Level,0,111,18,True,4,105338729,...,15782.959247,0.039626,NaN,True,95.0,2047.0,67.525570,100.0,3.48,111_18


In [15]:
# sort by EventTimeStamp
fault_diag_sorted = fault_diag.sort_values('EventTimeStamp').reset_index(drop = True)

In [37]:
fault_diag_sorted.head()

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,...,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,Speed,Throttle,TurboBoostPressure,spn_fmi
0,1211418,108604426,2000-03-18 19:14:10,High Voltage (Fuel Level),49,96,3,True,126,2015,...,37866.421934,0.673641,NaN,True,127.4,1279.0,0.000000,100.0,0.58,96_3
1,1211417,108604425,2000-03-18 19:14:10,High Voltage (Left Fuel Level Sensor),49,829,3,True,126,2015,...,37866.421934,0.673641,NaN,True,127.4,1279.0,0.000000,100.0,0.58,829_3
2,1211423,108608954,2000-03-19 03:58:23,Not Reporting Data Wheel Sensor ABS Axle 2 Right,11,792,7,True,14,1849,...,58344.246889,2.113383,32.0,True,82.4,1279.0,46.641670,100.0,14.50,792_7
3,1211432,108615304,2000-03-19 07:32:53,Low (Severity Medium) Transmission Air Tank Pr...,3,37,18,True,19,2283,...,7238.842569,1.056692,NaN,True,118.4,50175.0,0.145634,100.0,0.87,37_18
4,1211436,108620730,2000-03-19 08:40:03,High Voltage (Fuel Level),49,96,3,True,126,2034,...,44400.453468,0.528346,NaN,True,116.6,1279.0,0.000000,100.0,0.00,96_3


In [46]:
# to find the time at which the 80% event happens, use loc to get just derate rows
derates = fault_diag_sorted.loc[(fault_diag_sorted['spn'] == 5246)].reset_index(drop = True)
# len * .8 to find how many events to include in train
ts_08 = int(len(derates) * 0.8)
ts_06 = int(len(derates) * 0.6)
# get timestamp corresponding to that row number, everything at or before is train
split_08 = derates.loc[ts_08, 'EventTimeStamp']
split_06 = derates.loc[ts_06, 'EventTimeStamp']

In [47]:
# X_train
fault_diag_sorted.loc[fault_diag_sorted['EventTimeStamp'] <= split_06]

#X_val
fault_diag_sorted.loc[(fault_diag_sorted['EventTimeStamp'] > split_06) and (fault_diag_sorted['EventTimeStamp'] <= split_08)]

# X-test
fault_diag_sorted.loc[(fault_diag_sorted['EventTimeStamp'] > split_08)]

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,...,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,Speed,Throttle,TurboBoostPressure,spn_fmi
0,1211418,108604426,2000-03-18 19:14:10,High Voltage (Fuel Level),49,96,3,True,126,2015,...,37866.421934,0.673641,NaN,True,127.4,1279.0,0.000000,100.0,0.58,96_3
1,1211417,108604425,2000-03-18 19:14:10,High Voltage (Left Fuel Level Sensor),49,829,3,True,126,2015,...,37866.421934,0.673641,NaN,True,127.4,1279.0,0.000000,100.0,0.58,829_3
2,1211423,108608954,2000-03-19 03:58:23,Not Reporting Data Wheel Sensor ABS Axle 2 Right,11,792,7,True,14,1849,...,58344.246889,2.113383,32.0,True,82.4,1279.0,46.641670,100.0,14.50,792_7
3,1211432,108615304,2000-03-19 07:32:53,Low (Severity Medium) Transmission Air Tank Pr...,3,37,18,True,19,2283,...,7238.842569,1.056692,NaN,True,118.4,50175.0,0.145634,100.0,0.87,37_18
4,1211436,108620730,2000-03-19 08:40:03,High Voltage (Fuel Level),49,96,3,True,126,2034,...,44400.453468,0.528346,NaN,True,116.6,1279.0,0.000000,100.0,0.00,96_3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
333836,718356,14647492,2017-02-21 08:11:12,Incorrect Data J1939 Network #1 Primary Vehicl...,11,639,2,True,127,307,...,21954.810898,7.013790,32.0,True,125.6,1279.0,4.233091,0.0,5.51,639_2
333837,718349,14647332,2017-02-21 08:13:16,Incorrect Data Particulate Trap Outlet Pressure 1,0,3610,2,True,5,1816,...,26024.777617,0.858562,32.0,True,89.6,17407.0,0.000000,100.0,0.29,3610_2
333838,718350,14647356,2017-02-21 08:14:01,Abnormal Update Rate Wheel-Based Vehicle Speed,49,84,9,True,2,1703,...,16782.718378,0.000000,32.0,True,86.0,1279.0,158.447200,100.0,1.16,84_9
333839,718351,14647357,2017-02-21 08:14:03,Abnormal Frequency J1939 Network #2,49,1231,8,True,2,1703,...,16782.718378,1.703915,32.0,True,86.0,1279.0,158.447200,100.0,0.87,1231_8


In [ ]:
# modifying previously used model to use time-based split

# to find the time at which the 80% event happens, use loc to get just derate rows
derates = fault_diag_sorted.loc[(fault_diag_sorted['spn'] == 5246)].reset_index(drop = True)
# len * .8 to find how many events to include in train
ts_08 = int(len(derates) * 0.8)
ts_06 = int(len(derates) * 0.6)
# get timestamp corresponding to that row number, everything at or before is train
split_08 = derates.loc[ts_08, 'EventTimeStamp']
split_06 = derates.loc[ts_06, 'EventTimeStamp']



# take AcceleratorPedal, FuelLtd, DistanceLtd, FuelRate, Speed out of model and try it with cost threshold
variables = ['BarometricPressure', 'CruiseControlSetSpeed', 'EngineCoolantTemperature', 'EngineLoad', 'EngineOilPressure', 'EngineOilTemperature', 'EngineRpm', 'EngineTimeLtd', 
             'FuelLevel', 'IntakeManifoldTemperature', 'TurboBoostPressure', 'FuelTemperature', 'Throttle']

X = fault_diag[variables]
y = fault_diag['derate_full']

# train-test-validation split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, random_state = 321, train_size = 0.8)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, stratify = y_train, random_state = 321, train_size = 0.6/0.8)

# same model as before
# Impute missing values with the mean of each column
imputer = SimpleImputer(strategy = 'mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)
X_val_imputed = imputer.transform(X_val)

logreg = LogisticRegression(max_iter = 1000).fit(X_train_imputed, y_train)

# predicted probabilities for test data
y_prob = logreg.predict_proba(X_val_imputed)[:, 1]

# costs defined
cost_tp = 4000
cost_fp = 500

# decision threshold with cost as a factor
# threshold = cost_fn / (cost_fn + cost_fp)
threshold = 0.005

# probabilities converted to binary predictions based on new decision threshold
y_pred_adjusted = (y_prob >= threshold).astype(int)

# evaluation of adjusted predictions
print(f'Accuracy: {accuracy_score(y_test, y_pred_adjusted)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred_adjusted)}')
print(classification_report(y_test, y_pred_adjusted, zero_division = 0.0))

# confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred_adjusted)
print(f'Confusion matrix: {conf_matrix}')

# cost calculation
total_cost = conf_matrix[1, 1] * cost_tp - conf_matrix[0, 1] * cost_fp # 2nd column
print(f'Total cost: {total_cost}')

calculate cost if there was no model (so 2nd column in confusion matrix is 0, 0)
- 4000 * num derates in data

comparison:
- 4000 for each missed (lower left) + cost for false positive (upper right)
- difference is amount saved

quicker: just look at right side (calculate net benefit)
- saved 4000 for upper right and cost 500 for lower right
- upper right increases cost
- lower right decreases cost
- positive number = better than no model, negative means worse